# Init Kernel

In [ ]:
!python3 -m venv myenvCHAMPI

In [ ]:
!python -m ipykernel install --user --name=myenvCHAMPI

In [ ]:
#Windows
# myenvCHAMPI\Scripts\activate.bat
#Mac
!source myenvCHAMPI/bin/activate

Relancer la page + selectionner le nouvel env en haut à droite

In [ ]:
!pip install -r requirements.txt

# Import des packages

In [1]:
import numpy as np              # linear algebra
import pandas as pd             # data processing, CSV file I/O (e.g. pd.read_csv)
import os                       # files handling
import re
from PIL import Image
from random import randint, seed
from IPython.display import display
import matplotlib.pyplot as plt
import json
import warnings
import requests
import seaborn as sns
warnings.filterwarnings('ignore')

# Téléchargement des données

In [2]:
def load_json_data():
    
    # Créer une liste vide pour stocker les données JSON de chaque fichier
    data = []
    
    # Charger chaque fichier JSON et stocker ses données dans la liste de données
    for i in range(1,11):
        with open(f"C:/Users/legal/Documents/Personelle/Ecole/EICNAM/Semestre 4/EntreposageFouillesdeDonnees/TP_Note/TP_CHAMPI/json_files/observations_{i}.json") as f:
            json_data = json.load(f)
            if i==1:
                df = pd.DataFrame(json_data)
            else:
                df = df.append(json_data, ignore_index=True)
            
            new_cols = {}

    for i, gbif_info in enumerate(df['gbif_info']):
        # Vérification que la valeur est un dictionnaire
        if isinstance(gbif_info, dict):
            # Parcours de toutes les clés du dictionnaire dans la colonne "gbif_info"
            for col in set().union(*(d.keys() for d in [gbif_info])):
                # Création d'une nouvelle colonne avec le nom de la clé et une liste vide pour stocker les valeurs
                if col not in new_cols:
                    new_cols[col] = [None] * len(df.index)
                
                # Stockage de la valeur correspondante dans la liste de la nouvelle colonne
                new_cols[col][i] = gbif_info.get(col)

    # Fusion des nouvelles colonnes avec le dataframe principal
    df = df.join(pd.DataFrame(new_cols))
    #for i, image_url in enumerate(df['image_url']):
    #    pass
    return df
df= load_json_data()
df.head(3)

,observation,label,image_id,image_url,user,date,gbif_info,thumbnail,location,species,...,genus,speciesKey,canonicalName,genusKey,usageKey,rank,class,phylum,status,note
0,24671,Boletus nobilissimus,54745,http://mushroomobserver.org/images/320/54745,290,2009-08-24 02:54:15,"{'kingdom': 'Fungi', 'family': 'Boletaceae', '...",1,2259.0,Boletus nobilissimus,...,Boletus,6015253.0,Boletus nobilissimus,7241486.0,6015253.0,SPECIES,Agaricomycetes,Basidiomycota,ACCEPTED,None
1,24671,Boletus nobilissimus,54746,http://mushroomobserver.org/images/320/54746,290,2009-08-24 02:54:15,"{'kingdom': 'Fungi', 'family': 'Boletaceae', '...",0,2259.0,Boletus nobilissimus,...,Boletus,6015253.0,Boletus nobilissimus,7241486.0,6015253.0,SPECIES,Agaricomycetes,Basidiomycota,ACCEPTED,None
2,24671,Boletus nobilissimus,54747,http://mushroomobserver.org/images/320/54747,290,2009-08-24 02:54:15,"{'kingdom': 'Fungi', 'family': 'Boletaceae', '...",0,2259.0,Boletus nobilissimus,...,Boletus,6015253.0,Boletus nobilissimus,7241486.0,6015253.0,SPECIES,Agaricomycetes,Basidiomycota,ACCEPTED,None


On a un problème avec les urls.

Le Site a changé les chemins des images on passe de

 http://mushroomobserver.org/images/320/54751   

 https://images.mushroomobserver.org/320/54751.jpg

In [3]:
df['image_url'] = df['image_url'].replace('w*$', '.jpg', regex=True) # Ajout du jpg à la fin
df['image_url'] = df['image_url'].replace('mushroomobserver.org/images', 'images.mushroomobserver.org', regex=True) # Changement du chemin

# Selection des champignons avec le status ACCEPTED et confiance >0.95

In [4]:
df = df.loc[df['confidence'] > 0.95]
df = df.loc[df['status']=='ACCEPTED']

# Champignon Francais

In [5]:
listfrance = ["Agaricus arvensis","Agaricus augustus","Agaricus campestris","Agaricus silvicola","Agaricus xanthoderma","Agrocybe cylindracea","Albatrellus ovinus","Aleuria aurantia","Amanita caesarea","Amanita citrina","Amanita fulva","Amanita junquillea","Amanita gemmata","Amanita muscaria","Amanita ovoidea","Amanita pantherina","Amanita phalloides","Amanita rubescens","Amanita spissa","Amanita vaginata","Amanita verna","Amanita virosa","Ampulloclitocybe clavipes","Anthurus archeri","Armillaria mellea","Auricularia auricula-judae","Boletus aereus","Boletus aestivalis","Boletus appendiculatus","Boletus calopus","Boletus edulis","Boletus erythropus","Boletus luridus","Boletus pinophilus","Boletus queletii","Boletus satanas","Boletus rhodopurpureus","Boletus aereus","Boletus edulis","Boletus pinophilus","Boletus aestivalis","Boletus aereus","Calocera viscosa","Calocybe gambosa","Calvatia excipuliformis","Calvatia gigantea","Cantharellus cibarius","Chalciporus piperatus","Chlorophyllum rhacodes","Chroogomphus rutilus","Clathrus ruber","Clavariadelphus pistillaris","Clavariadelphus truncatus","Clavulina cristata","Clavulina rugosa","Clitocybe amoenolens","Clitocybe connata","Clitocybe nebularis","Clitocybe odora","Clitopilus prunulus","Coprinellus micaceus","Coprinopsis atramentaria","Coprinopsis picacea","Coprinus comatus","Coprinus disseminatus","Cortinarius alboviolaceus","Cortinarius camphoratus","Cortinarius caperatus","Cortinarius cinnamomeus","Cortinarius orellanus","Cortinarius praestans","Cortinarius speciosissimus","Cortinarius splendens","Cortinarius triumphans","Cortinarius varius","Cortinarius violaceus","Craterellus cinereus","Craterellus cornucopioides","Craterellus lutescens","Craterellus tubaeformis","Daedaleopsis tricolor","Dacrymyces stillatus","Disciotis venosa","Dendropolyporus umbellatus","Entoloma clypeatum","Entoloma sinuatum","Entoloma nitidum","Fuligo septica","Flammulina velutipes","Fistulina hepatica","Galerina marginata","Geastrum fimbriatum","Geastrum triplex","Gomphidius glutinosus","Gomphidius roseus","Gomphus clavatus","Grifola frondosa","Gymnopilus penetrans","Gymnopilus spectabilis","Gymnopus dryophilus","Gymnopus peronatus","Gyromitra esculenta","Gyroporus castaneus","Gyroporus cyanescens","Hebeloma crustuliniforme","Hebeloma laterinum","Hebeloma radicosum","Helvella crispa","Helvella lacunosa","Helvella macropus","Hydnum repandum","Hydnum rufescens","Hygrocybe conica","Hygrocybe pratensis","Hygrocybe virginea","Hygrophoropsis aurantiaca","Hygrophorus agathosmus","Hygrophorus eburneus","Hygrophorus latitabundus","Hygrophorus nemoreus","Hygrophorus russula","Hypholoma fasciculare","Hypholoma lateritium","Infundibulicybe geotropa","Infundibulicybe gibba","Inocybe fastigiata","Inocybe rimosa","Inocybe geophylla","Inocybe patouillardii","Inocybe erubescens","Kuehneromyces mutabilis","Laccaria amethystina","Laccaria laccata","Lactarius blennius","Lactarius camphoratus","Lactarius chrysorrheus","Lactarius deliciosus","Lactarius deterrimus","Lactarius lignyotus","Lactarius necator","Lactarius piperatus","Lactarius rufus","Lactarius sanguifluus","Lactarius torminosus","Lactarius vellereus","Lactarius volemus","Laetiporus sulphureus","Leccinum aurantiacum","Leccinum crocipodium","Leccinum lepidum","Leccinum scabrum","Leccinum versipelle","Lepiota brunneoincarnata","Lepiota clypeolaria","Lepiota cristata","Lepista flaccida","Lepista nuda","Lepista saeva","Lepista sordida","Leucoagaricus leucothites","Lycoperdon echinatum","Lycoperdon perlatum","Lyophyllum decastes","Macrolepiota mastoidea","Macrolepiota procera","Marasmius oreades","Megacollybia platyphylla","Morchella elata","Morchella esculenta","Morganella pyriformis","Mutinus caninus","Mycena polygramma","Mycena pura","Mycena rosea","Otidea onotica","Omphalotus olearius","Oudemansiella mucida","Paxillus involutus","Phaeolepiota aurea","Phallus impudicus","Pholiota squarrosa","Pleurotus cornucopiae","Pleurotus eryngii","Pleurotus ostreatus","Pluteus cervinus","Pluteus leoninus","Polyporus squamosus","Psathyrella candolleana","Psathyrella piluliformis","Pseudoclitocybe cyathiformis","Pseudohydnum gelatinosum","Psilocybe semilanceata","Ramaria aurea","Ramaria botrytis","Ramaria formosa","Ramaria stricta","Rhodocollybia butyracea","Russula amara","Russula badia","Russula cutefracta","Russula cyanoxantha","Russula emetica","Russula olivacea","Russula queletii","Russula risigallina","Russula chamaeleontina","Russula sanguinaria","Russula sardonia","Russula drimeia","Russula vesca","Russula virescens","Russula xerampelina","Sarcoscypha coccinea","Schizophyllum commune","Scleroderma citrinum","Scleroderma verrucosum","Sparassis crispa","Strobilomyces strobilaceus","Stropharia aeruginosa","Suillus bovinus","Suillus cavipes","Suillus granulatus","Suillus grevillei","Suillus luteus","Tapinella atrotomentosa","Tremiscus helvelloides","Tricholoma album","Tricholoma equestre","Tricholoma pardinum","Tricholoma portentosum","Tricholoma scalpturatum","Tricholoma sciodes","Tricholoma sulphureum","Tricholoma terreum","Tricholoma virgatum","Tricholomopsis rutilans","Tuber melanosporum","Tylopilus felleus","Volvariella gloiocephala","Xerocomus badius","Xerocomus chrysenteron","Xerula radicata"]

In [6]:
dfchampi = df.loc[df['label'].isin(listfrance)]

# Selection TOP 10

In [7]:
counts=dfchampi["label"].value_counts()
selected = dfchampi[dfchampi["label"].isin(counts.nlargest(10).index)] 
# Afficher les champignons sélectionnés
selected.shape

(9448, 32)

# Téléchargement des images

In [8]:
'''
# créer un dossier pour stocker les images
if not os.path.exists("images"):
    os.makedirs("images")

# itérer sur chaque ligne du dataset
total_images = len(selected)
percent_step = 5
current_percent = percent_step
for index, row in selected.iterrows():
    # récupérer l'URL de l'image
    image_url = row["image_url"]
    # extraire l'ID de l'image du nom du fichier
    image_id = row["image_id"]
    # récupérer le contenu de l'image
    response = requests.get(image_url)
    # écrire le contenu de l'image dans un fichier local
    with open(f"images/{image_id}.jpg", "wb") as f:
        f.write(response.content)
    # afficher la progression en pourcentage
    current_percent = (index + 1) * 100 // total_images
    if current_percent >= percent_step:
        print(f"{current_percent}% des images téléchargées")
        percent_step += 5
'''

'\n# créer un dossier pour stocker les images\nif not os.path.exists("images"):\n    os.makedirs("images")\n\n# itérer sur chaque ligne du dataset\ntotal_images = len(selected)\npercent_step = 5\ncurrent_percent = percent_step\nfor index, row in selected.iterrows():\n    # récupérer l\'URL de l\'image\n    image_url = row["image_url"]\n    # extraire l\'ID de l\'image du nom du fichier\n    image_id = row["image_id"]\n    # récupérer le contenu de l\'image\n    response = requests.get(image_url)\n    # écrire le contenu de l\'image dans un fichier local\n    with open(f"images/{image_id}.jpg", "wb") as f:\n        f.write(response.content)\n    # afficher la progression en pourcentage\n    current_percent = (index + 1) * 100 // total_images\n    if current_percent >= percent_step:\n        print(f"{current_percent}% des images téléchargées")\n        percent_step += 5\n'

In [9]:
def load_jpeg_images(path):
    image_files = [f for f in os.listdir(path) if f.endswith('.jpg') or f.endswith('.jpeg')]
    images = []
    ids=[]
    for file in image_files:
        ids.append(os.path.splitext(file)[0])
        with Image.open(os.path.join(path, file)) as image:
            image_data = np.array(image)
            images.append(image_data)
    return pd.DataFrame({"ids":np.array(ids) , "img":np.array(images)})
    
df_images=load_jpeg_images("images/")

In [12]:
if not os.path.exists("dataframe"):
    os.makedirs("dataframe")

# Export en csv de notre dataframe

In [15]:
selected.to_csv("dataframe/champignons.csv")

In [16]:
df_images.to_csv("dataframe/df_images.csv")